# Populating the LUSID instrument master from different data sources

In this training module we'll see how to use LUSID to perform the following task:

**<div align="center">As a data manager, I want to populate the LUSID instrument master with a mix of Equities, Bonds, Futures and FxForwards. The data is sourced from two systems, each with its own taxonomy and market identifiers. I want to a see a consolidated view for each instrument in LUSID, but retain the original format and lineage of the data.</div>**

In [1]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
#display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.9137.0,0.5.2829,"{'relation': 'RequestLogs', 'href': 'http://ja..."


In [2]:
# Create a scope and code to segregate data in this module from other modules
module_scope = "FBNUniversity"
module_code = "Module-2-1"
print(f"'{module_scope}\{module_code}' scope and code created.")

'FBNUniversity\Module-2-1' scope and code created.


## 1. Examining the source files

In [3]:
# Read security information into pandas dataframe from VendorA, showing all the decimal places for the bond coupon
with pd.option_context('display.float_format', '{:,.3f}'.format):
    VendorA_df = pd.read_csv("VendorA.csv", keep_default_na = False)
    display(VendorA_df)

,instrument_name,internal_id,currency,isin,figi,country_issue,market_sector,coupon%,maturity_date
0,BP PLC,imd_43535553,GBP,GB0007980591,BBG000C05BD1,United Kingdom,equity,0.000,
1,UK Gilt 0.375 10/22/26,imd_34994599,GBP,GB00BNNGP668,BBG00ZF1T9P5,united_kingdom,bond,0.375,2026-10-22
2,EURO-BUND FUTURE Jun22,imd_34588699,EUR,DE0009652644,BBG012FFPKW7,,future,0.000,2022-06-30
3,EURUSD 6M FWD,imd_34876136,USD,,,United States,fxforward,0.000,2022-07-01


In [4]:
# Read security information into pandas dataframe from VendorA, showing all the decimal places for the bond coupon
with pd.option_context('display.float_format', '{:,.3f}'.format):
    VendorB_df = pd.read_csv("VendorB.csv", keep_default_na = False)
    display(VendorB_df)

,instrument_name,internal_id,figi,ticker,currency,origin,market_sector,coupon%,maturity_date
0,BP,imd_43535553,BBG000C05BD1,BP/LN,GBP,GB,equity,0.000,
1,UKT 0 ⅜ 10/22/26,imd_34994599,BBG00ZF1T9P5,UKT 0.375 10/22/26,GBP,Great Britain,bond,0.375,2026-10-22
2,Euro-Bund Futures (FGBL),imd_34588699,BBG012FFPKW7,RXM2,EUR,DE,future,0.000,2022-06-30
3,EUR/USD 6M 20220101,imd_34876136,,,USD,usa,fxforward,0.000,2022-07-01


## 2. Mastering instruments in LUSID

An instrument must have at least one unique identifier type, such as Figi or ClientInternal. We can map the 'figi' column to Figi and the 'internal_id' column to ClientInternal, to give two unique identifiers per instrument. Because the values in these columns are the same for both vendors, LUSID automatically merges equivalent records to the same instrument. 

We can also map appropriate columns in either file to the Isin and Ticker non-unique identifiers for each instrument. In addition, LUSID automatically generates a third unique identifier for each successfully-mastered instrument: a LUID.

Note a bond coupon must be expressed as a number rather than a percentage, so the UK bond paying 0.375% is defined as 0.00375.

Note also we'll upload the instruments to a custom instrument scope, to segregate the data from currencies and other instruments in the `default` scope.

In [5]:
# Obtain the LUSID Instruments API
instruments_api = api_factory.build(la.InstrumentsApi)

# Capture the set of LUIDs automatically generated when instruments are mastered, for later use
instrument_luids = set()

# Create a convenience function to call for each vendor dataframe
def create_and_upsert_instruments(vendor_dataframe):

    # Create a dictionary of instrument definitions
    definitions = {}

    # Iterate over each row in the vendor dataframe
    for index, security in vendor_dataframe.iterrows():

        # Map possible identifier columns to case-sensitive LUSID identifier type names
        identifier_columns = [
            ("isin", "Isin"), 
            ("figi", "Figi"), 
            ("ticker", "Ticker"),
            ("internal_id", "ClientInternal")
        ]
        # Create instrument identifiers
        identifiers = {}
        for identifier in identifier_columns:
             # Test whether identifier column exists and (if so) is not NaN or empty
            if (identifier[0] in security) and pd.notna(security[identifier[0]]) and (security[identifier[0]] != u''):
                identifiers[identifier[1]] = lm.InstrumentIdValue(
                    value = security[identifier[0]]
            )

        # Model equities
        if security["market_sector"] == "equity":
            # Create definitions
            definitions[security["instrument_name"]] = lm.InstrumentDefinition(
                name = security["instrument_name"],
                identifiers = identifiers,
                definition = lm.Equity(
                    instrument_type = "Equity",
                    dom_ccy = "GBP",
                    identifiers = {}
                )
            )
        # Model bonds
        elif security["market_sector"] == "bond":
            definitions[security["instrument_name"]] = lm.InstrumentDefinition(
                name = security["instrument_name"],
                identifiers = identifiers,
                definition = lm.Bond(
                    instrument_type = "Bond",
                    start_date = "2021-01-01",
                    maturity_date = security["maturity_date"],
                    dom_ccy = security["currency"],
                    flow_conventions = lm.FlowConventions(
                        currency = security["currency"],
                        payment_frequency = "6M",
                        day_count_convention = "ActualActual",
                        roll_convention = "NoAdjustment",
                        payment_calendars = [],
                        reset_calendars = [],
                        settle_days = 0,
                        reset_days = 0
                    ),
                    principal = 1,
                    # Specified as a number rather than percentage, eg. so a bond paying 0.375% is specified as 0.00375
                    coupon_rate = security["coupon%"] / 100
                )
            )
        # Model futures
        elif security["market_sector"] == "future":
            definitions[security["instrument_name"]] = lm.InstrumentDefinition(
                name = security["instrument_name"],
                identifiers = identifiers,
                definition = lm.Future(
                    instrument_type = "Future",
                    start_date = "2022-01-01",
                    maturity_date = security["maturity_date"],
                    identifiers = {},
                    contract_details = lm.FuturesContractDetails(
                        dom_ccy = security["currency"],
                        contract_code = "FGBL",
                        contract_month = "M",
                        contract_size = 100000,
                        convention = "ActualActual",
                        country = "DE",
                        description = security["instrument_name"],
                        exchange_code = "EUREX",
                        exchange_name = "Eurex",
                        ticker_step = 0.01,
                        unit_value = 10,
                    ),
                    contracts = 1,
                    underlying = lm.ExoticInstrument(
                        instrument_format = lm.InstrumentDefinitionFormat(
                            "custom", "custom", "0.0.0"
                        ),
                        content = "{}",
                        instrument_type = "ExoticInstrument",
                    )
                )
            )
        # Model FxForwards
        elif security["market_sector"] == "fxforward":
            definitions[security["instrument_name"]] = lm.InstrumentDefinition(
                name = security["instrument_name"],
                identifiers = identifiers,
                definition = lm.FxForward(
                    instrument_type = "FxForward",
                    start_date = "2022-01-01",
                    maturity_date = security["maturity_date"],
                    dom_ccy = "EUR",
                    dom_amount = 1000000,
                    fgn_ccy = security["currency"],
                    fgn_amount = -1215520.25
                )
            )
    
    # Upsert instruments to LUSID
    upsert_instruments_response = instruments_api.upsert_instruments(
        request_body = definitions,
        # Master the instruments in a custom scope
        scope = f"{module_scope}{module_code}",
    )
    
    # Capture LUIDs
    for instr in list(upsert_instruments_response.values.values()):
        instrument_luids.add(instr.lusid_instrument_id)

    # Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
    upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
    display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])
    
    
# The first time we call this function, instruments are created
create_and_upsert_instruments(VendorA_df)
# The second time, instruments are updated (the name changes but the LUID stays the same)
create_and_upsert_instruments(VendorB_df)

,name,lusid_instrument_id
0,UK Gilt 0.375 10/22/26,LUID_00003D96
1,BP PLC,LUID_00003D97
2,EURUSD 6M FWD,LUID_00003D98
3,EURO-BUND FUTURE Jun22,LUID_00003D99


,name,lusid_instrument_id
0,Euro-Bund Futures (FGBL),LUID_00003D99
1,EUR/USD 6M 20220101,LUID_00003D98
2,UKT 0 ⅜ 10/22/26,LUID_00003D96
3,BP,LUID_00003D97


## 3. Adding properties to instruments

Instrument definitions do not have fields to store information about domicile.

We can add as many properties to instruments as we like, to store extra information. We'll add two properties to each instrument, to store the data from VendorA ('country_issue') and VendorB ('origin') separately and in its original form.

### 3.1 Creating property definitions

The first task is to create a property definition for each property.

In [6]:
# Obtain the LUSID Property Definition API
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)

# Create a convenience function to call for each property definition
def create_and_upsert_property_definition(property_scope, property_code):
    
    # Create property definition specific to instruments, with a unique scope and code
    property_definition = lm.CreatePropertyDefinitionRequest(
        domain = "Instrument",
        scope = property_scope,
        code = property_code,
        display_name = property_code,
        data_type_id = lm.ResourceId(
            scope = "system",
            code = "string"
        )
    )
    
    # Upsert property definition to LUSID
    try:
        upsert_property_definition_response = property_definition_api.create_property_definition(
            create_property_definition_request = property_definition
        )
        print(f"Property definition created with the following key: {upsert_property_definition_response.key}")
    except lu.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property definition with the following key already exists: {property_definition.domain}/{property_definition.scope}/{property_definition.code}"
            )
    
# Create a property definition representing 'country_issue' from VendorA
create_and_upsert_property_definition("VendorA","country_issue")
# Create a property definition representing 'origin' from VendorB
create_and_upsert_property_definition("VendorB","origin")

INFO:root:Property definition with the following key already exists: Instrument/VendorA/country_issue
INFO:root:Property definition with the following key already exists: Instrument/VendorB/origin


### 3.2 Adding properties to instruments

We can now iterate over the dataframe from each vendor, adding properties with appropriate values to each instrument in turn.

Note we must specify the custom instrument scope in which the instruments are mastered (since they do not exist in the `default` scope),

In [7]:
# Create a convenience function to call for each vendor dataframe
def add_properties_to_instruments(vendor_dataframe, property_scope, property_code):   
    property_request = [
        lm.UpsertInstrumentPropertyRequest(
            identifier_type = "ClientInternal",
            identifier = security["internal_id"],
            properties = [
                lm.ModelProperty(
                    key = f"Instrument/{property_scope}/{property_code}",
                    value = lm.PropertyValue(
                        label_value = security[property_code]
                        )
                )
            ]
        )
        for index, security in vendor_dataframe.iterrows()
    ]   
    upsert_properties_response = instruments_api.upsert_instruments_properties(
        # Specify the custom instrument scope
        scope = f"{module_scope}{module_code}",
        upsert_instrument_property_request = property_request,
    )
    
    print(f"Properties from {property_scope} updated at {(str(upsert_properties_response.as_at_date))}")
    
    
# Add 'country_issue' properties from VendorA to instruments    
add_properties_to_instruments(VendorA_df, "VendorA", "country_issue")
# Add 'origin' properties from VendorB to instruments 
add_properties_to_instruments(VendorB_df, "VendorB", "origin")

Properties from VendorA updated at 2022-05-05 09:29:20.436186+00:00
Properties from VendorB updated at 2022-05-05 09:29:20.676632+00:00


### 3.3 Confirming properties have been added

We must explicitly request LUSID to 'decorate' properties onto instruments when retrieving the latter, using each property's 3-stage key. For example, we can use our set of LUIDs captured earlier to specify the instruments to retrieve, and request the properties to decorate using the `property_keys` parameter.

Note we must again specify the custom instrument scope in which the instruments are mastered.

In [8]:
# Retrieve our instruments, decorating particular properties onto response
get_instruments_response = instruments_api.get_instruments(
    # Specify the custom instrument scope
    scope = f"{module_scope}{module_code}",
    identifier_type = "LusidInstrumentId", 
    request_body = list(instrument_luids),
    property_keys = ["Instrument/VendorA/country_issue", "Instrument/VendorB/origin"]
)

#Transform API response to a dataframe and show the properties for each instrument
get_instruments_response_df = lusid_response_to_data_frame(list(get_instruments_response.values.values()))
display(get_instruments_response_df[["name", 
                                     "properties.0.key", 
                                     "properties.0.value.label_value",
                                     "properties.1.key",
                                     "properties.1.value.label_value",
                                    ]])

,name,properties.0.key,properties.0.value.label_value,properties.1.key,properties.1.value.label_value
0,EUR/USD 6M 20220101,Instrument/VendorA/country_issue,United States,Instrument/VendorB/origin,usa
1,BP,Instrument/VendorA/country_issue,United Kingdom,Instrument/VendorB/origin,GB
2,UKT 0 ⅜ 10/22/26,Instrument/VendorA/country_issue,united_kingdom,Instrument/VendorB/origin,Great Britain
3,Euro-Bund Futures (FGBL),Instrument/VendorA/country_issue,,Instrument/VendorB/origin,DE


## 4. Creating a derived property

### 4.1 Specifying a derivation formula

We want our derived property to:

1. Merge the 'country_issue' and 'origin' properties to account for missing values (using the `coalesce` function), and then
2. Normalise the values to a standard set (using the `map` function).

In [9]:
# The coalese function prefers values from the first property supplied ('country_issue'). Values from the second
# property ('origin') are only used if the first has no values. The map function then maps 
# values to a standard set, for example 'DE' to 'Germany'
derived_property_formula = """map(coalesce(Properties[Instrument/VendorA/country_issue], 
                                           Properties[Instrument/VendorB/origin], 'Unknown'): 
                                                'United Kingdom'='UK', 'united_kingdom'='UK', 
                                                'Great Britain'='UK', 'GB'='UK', 'DE'='Germany',
                                                'United States'='USA', 'usa'='USA', default='Unknown')"""

### 4.2 Creating a derived property definition

In [10]:
derived_property_definition = lm.CreateDerivedPropertyDefinitionRequest(
    domain = "Instrument",
    scope = "Derived",
    code = "Domicile",
    display_name = "Domicile",
    data_type_id = lm.ResourceId(code="string", scope="system"),
    derivation_formula = derived_property_formula,
)

# Upsert derived property definition to LUSID
try:
    derived_property_definition_response = property_definition_api.create_derived_property_definition(
        create_derived_property_definition_request = derived_property_definition
    )
    print(f"Derived property definition created with the following key: {derived_property_definition_response.key}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PropertyAlreadyExists":
        logging.info(
            f"Derived property definition with the following key already exists:{derived_property_definition.domain}/{derived_property_definition.scope}/{derived_property_definition.code}"
        )

INFO:root:Derived property definition with the following key already exists:Instrument/Derived/Domicile


### 4.3 Confirming the derived property can be calculated

We don't have to explicitly add derived properties to instruments; rather, LUSID calculates values on-the-fly when retrieving instruments (see the `property_keys` parameter for the request to decorate the derived property onto retrieved instruments). We must still specify the custom instrument scope in which the instruments are mastered.

In [11]:
# Retrieve our instruments, decorating particular properties onto response
get_instruments_response = instruments_api.get_instruments(
    # Specify the custom instrument scope
    scope = f"{module_scope}{module_code}",
    identifier_type = "LusidInstrumentId",
    request_body = list(instrument_luids),
    property_keys = ["Instrument/VendorA/country_issue", "Instrument/VendorB/origin", "Instrument/Derived/Domicile"]
)

# Transform API response to a dataframe and show the properties for each instrument
get_instruments_response_df=lusid_response_to_data_frame(list(get_instruments_response.values.values()))
display(get_instruments_response_df[["name", 
                                     "properties.0.key", 
                                     "properties.0.value.label_value",
                                     "properties.1.key",
                                     "properties.1.value.label_value",
                                     "properties.2.key",
                                     "properties.2.value.label_value",
                                    ]])

,name,properties.0.key,properties.0.value.label_value,properties.1.key,properties.1.value.label_value,properties.2.key,properties.2.value.label_value
0,BP,Instrument/VendorA/country_issue,United Kingdom,Instrument/VendorB/origin,GB,Instrument/Derived/Domicile,UK
1,EUR/USD 6M 20220101,Instrument/VendorA/country_issue,United States,Instrument/VendorB/origin,usa,Instrument/Derived/Domicile,USA
2,UKT 0 ⅜ 10/22/26,Instrument/VendorA/country_issue,united_kingdom,Instrument/VendorB/origin,Great Britain,Instrument/Derived/Domicile,UK
3,Euro-Bund Futures (FGBL),Instrument/VendorA/country_issue,,Instrument/VendorB/origin,DE,Instrument/Derived/Domicile,Germany
